![](https://www.calltouch.ru/upload/medialibrary/a67/a67539dadca3279f6c8943ce1fbafb8b.jpg)

# <center> Компьютер говорит «Нет» </center>

**client_id** - идентификатор клиента

**education** - уровень образования

**sex** - пол заемщика

**age** - возраст заемщика

**car** - флаг наличия автомобиля

**car_type** - флаг автомобиля иномарки

**decline_app_cnt** - количество отказанных прошлых заявок

**good_work** - флаг наличия “хорошей” работы

**bki_request_cnt** - количество запросов в БКИ

**home_address** - категоризатор домашнего адреса

**work_address** - категоризатор рабочего адреса

**income** - доход заемщика

**foreign_passport** - наличие загранпаспорта

**sna** - связь заемщика с клиентами банка

**first_time** - давность наличия информации о заемщике

**score_bki** - скоринговый балл по данным из БКИ

**region_rating** - рейтинг региона

**app_date** - дата подачи заявки

**default** - флаг дефолта по кредиту

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn import preprocessing
from sklearn.preprocessing import (
    OneHotEncoder, StandardScaler,
    RobustScaler, MinMaxScaler
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix,
    mean_squared_error, f1_score, accuracy_score,
    precision_score, recall_score,
    roc_curve, roc_auc_score
)
from sklearn.cluster import DBSCAN
from scipy import stats

import optuna

import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

from imblearn.under_sampling import RandomUnderSampler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install optuna

In [ ]:
# функция уникальных значений каждой переменной
def meanings(df):
    for each in df.columns:
        print(f'Значения в колонке {each}:', df[each].unique())

In [ ]:
# функция приведения переменных типа 'object' к 'int'
def label_encoding(df):
    le = preprocessing.LabelEncoder()
    for each in df.select_dtypes('object').columns:
        # добавлена ветвь, так как kaggle не категоризирует ветвь с Nan
        if df[each].isnull().values.sum() != 0:
            df[each] = df[each].fillna('Nan')
        else:
            ''
        df[each] = df[each].sort_values(ascending=True)
        df[each] = le.fit_transform(df[each].values)
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        print(f'Словарь для параметра {each}:', le_dict)

In [ ]:
# построение группы боксплотов
def boxplot_out(df, columns):
    fig = plt.figure(figsize=[30, 8])
    count = 1
    for each in columns:
        ax = fig.add_subplot(1, 5, count)
        sns.boxplot(y=df[each], data=df[each], orient='v')
        ax.set_title(label=each, fontdict={'fontsize': 12})
        ax.set(ylabel='')
        plt.subplots_adjust(wspace=1)
        count += 1
    plt.show()

In [ ]:
# визуализация выбросов
def graph_out(df, columns):
    fig = plt.figure(figsize=[30, 4])
    count = 1
    for each in columns:
        ax = fig.add_subplot(1, 5, count)
        plt.title(each)

        perc25 = df[each].quantile(0.25)
        perc75 = df[each].quantile(0.75)
        IQR = perc75 - perc25

        df[each].loc[df[each].between(
            perc25 - 1.5*IQR, perc75 + 1.5*IQR)].hist(label='IQR')
        df[each].loc[(df[each] < perc25 - 1.5*IQR) |
                     (df[each] > perc75 + 1.5*IQR)].hist(label='выбросы')

        count += 1

        plt.legend()

In [ ]:
# статистика по выбросам
def outliers(df, columns):
    for each in columns:
        norm_dist = np.random.choice(
            df[each].dropna().values, 5000).reshape(-1, 1)
        db = DBSCAN(eps=0.5, min_samples=10).fit(norm_dist)
        out_db = len(np.where(db.labels_ == -1)[0])
        out_db_perc = round(100*out_db / df.shape[0], 2)
        print(
            f'Количество выбросов у {each} по DBSCAN: {out_db}, {out_db_perc}% выборки.')

        mean = np.mean(df[each])
        std = np.std(df[each])
        z_score = [(x-mean)/std for x in df[each]]
        out_z = len(np.where(np.abs(z_score) > 3)[0])
        out_z_perc = round(100*out_z / df.shape[0], 2)
        print(
            f'Количество выбросов у {each} по Z-score: {out_z}, {out_z_perc}% выборки.')

In [ ]:
# группа гистограмм
def hist_type(df, columns):
    fig = plt.figure(figsize=[30, 70])
    count = 1
    for each in columns:
        ax = fig.add_subplot(16, 5, count)
        sns.countplot(x=each, data=df)
        count += 1

In [ ]:
# удаление шума по z_score
def del_outliers(df, columns):
    for each in columns:
        mean = np.mean(df[each])
        std = np.std(df[each])
        moda = df[each].mode()
        z_score = [(x-mean)/std for x in df[each]]
        out_z = np.where(np.abs(z_score) > 3)[0]
        for i in out_z:
            df[each][i] = moda

In [ ]:
# проверка разбивки данных на сбалансированность
def visualize_train_valid_counts(init_data, train, valid, target, ylim):
    x = np.array([0, 1])
    width = 0.2

    plt.figure(figsize=(15, 8))
    ax = plt.subplot(111)

    classes = list(init_data[target].value_counts().index)

    ax.bar(x - width, list(init_data[target].value_counts()
           [classes]), width, color='r', label='Исходные данные')
    ax.bar(x, list(train[target].value_counts()[classes]),
           width, color='g', label='Тренировочная выборка')
    ax.bar(x, list(valid[target].value_counts()[classes]), width, bottom=list(
        train[target].value_counts()[classes]), color='b', label='Валидационная выборка')

    ax.set_ylim([0, ylim])
    plt.xticks(x - width / 2, classes, fontsize=20)
    plt.yticks(fontsize=15)
    plt.ylabel('Кол-во примеров', fontsize=20)
    plt.minorticks_on()
    plt.grid(which='major', color='r')
    plt.grid(which='minor', linestyle=':', color='k')
    plt.legend(fontsize=15)

In [ ]:
# метрики для модели
def print_logisitc_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'accuracy = {acc:.3f}, precision = {prec:.3f}')
    print(f'recall = {rec:.3f}, F1-score = {f1:.3f}')

In [ ]:
# построение ROC AUC кривой
def rocauc(y_true, y_pred):
    roc_auc = roc_auc_score(y_true, y_pred)
    fpr, tpr, label = roc_curve(y_true, y_pred)
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label='Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# функция определения оптимальных параметров
def best_pred(X_train, y_train, X_valid):
    # запуск GridSearch на небольшом кол-ве итераций max_iter=50 и с достаточно большой дельтой останова tol1e-3
    # чтобы получить оптимальные параметры модели в первом приближении

    model = LogisticRegression(solver='liblinear')

    iter_ = 1000
    epsilon_stop = 1e-3

    param_grid = [
        {'penalty': ['l1'],
         'solver': ['liblinear', 'lbfgs'],
         'class_weight':['None', 'balanced'],
         'multi_class': ['auto', 'ovr'],
         'max_iter':[iter_],
         'tol':[epsilon_stop]},
        {'penalty': ['l2'],
         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
         'class_weight':['None', 'balanced'],
         'multi_class': ['auto', 'ovr'],
         'max_iter':[iter_],
         'tol':[epsilon_stop]},
        {'penalty': ['None'],
         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
         'class_weight':['None', 'balanced'],
         'multi_class': ['auto', 'ovr'],
         'max_iter':[iter_],
         'tol':[epsilon_stop]},
    ]

# model  - модель логистической регрессии
    gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
    gridsearch.fit(X_train, y_train)
    model = gridsearch.best_estimator_

# прогноз на полученных параметрах
    y_predict = gridsearch.predict_proba(X_valid)

# печатаем параметры
    best_parameters = model.get_params()
    for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

    return y_predict

In [ ]:
DATA_DIR = '/kaggle/input/sf-scoring/'
train = pd.read_csv(DATA_DIR+'/train.csv')
test = pd.read_csv(DATA_DIR+'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
train.info()

In [ ]:
pandas_profiling.ProfileReport(train)

### Дублей нет. Переменнные нуждаются в нормализации и стандартизации. Пропуски есть только у переменой "образование". Неинформативным признаком являются client_id. Дублирующие признаки car, car_type; home_address, work_address; sna, first_time. 

In [ ]:
pandas_profiling.ProfileReport(test)

In [ ]:
# объединение признаков трейн и тест в один датасет
train['sample'] = 1  # помечаем где у нас трейн
test['sample'] = 0  # помечаем где у нас тест
# зануление в тесте Rating, как целевого показателя для прогнозирования
test['default'] = 0

data = test.append(train, sort=False).reset_index(drop=True)  # объединяем

In [ ]:
meanings(data)

### Анализ app_date

In [ ]:
data.app_date = pd.to_datetime(data.app_date, format='%d%b%Y')

In [ ]:
data.app_date.describe()

В выгрузке данные за первых 4 месяца 2014 года

In [ ]:
data['app_date'] = data['app_date'].dt.day_of_week
data['app_date'].value_counts()

В будние дни (индексы 0-4) больше подают заявок, чем в выходные (индексы 5,6)

#### Удаление неинформативных переменных и разбиение остальных перемнных на классы

client_id перемнная с уникальными значениями

In [ ]:
data.drop(['client_id'], axis=1, inplace=True)

In [ ]:
# бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'foreign_passport', 'good_work']
# категориальные переменные
cat_cols = ['education', 'home_address', 'work_address',
            'sna', 'first_time', 'region_rating', 'app_date']
# числовые переменные
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt', 'score_bki']

In [ ]:
data.default.hist()

Данные не сбалансированы, поэтому для принятие решения в данной задаче классификации корректнее использовать confusion matrix, precision, recall, F1-score, ROC-кривые, чем accuracy

## Работа с непрерывными переменными

In [ ]:
fig = plt.figure(figsize=[60, 30])
count = 1
for each in num_cols:
    ax = fig.add_subplot(2, 3, count)
    sns.histplot(data[each], kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

Нормальное распределение имеет только переменная score_bki. У других параметров большая ассиметрия

#### Изображение непрерывных переменных после логарифмирования

In [ ]:
fig = plt.figure(figsize=[60, 30])
count = 1
for each in num_cols[:-1]:
    ax = fig.add_subplot(2, 3, count)
    sns.histplot(np.log(data[each]+1), kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

#### Изображение непрерывных переменных после нормализации Бокса-Кокса

In [ ]:
fig = plt.figure(figsize=[60, 30])
count = 1
for each in num_cols[:-1]:
    ax = fig.add_subplot(2, 3, count)
    sns.histplot(stats.boxcox(
        data[each][data[each] > 0].dropna())[0], kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

#### Изображение непрерывных переменных после нормализации Йео-Джонсона

In [ ]:
fig = plt.figure(figsize=[60, 30])
count = 1
for each in num_cols:
    ax = fig.add_subplot(2, 3, count)
    sns.histplot(stats.yeojohnson(data[each])[0], kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

Преобразования Йео-Джонсона и Бокса-Кокса не подходят, так как дают нулевые значения и в дата фрейме появляются Nan, поэтому данные подвергнуться логарифмированию.

### Выбросы

In [ ]:
boxplot_out(data, num_cols)

In [ ]:
graph_out(data, num_cols)

In [ ]:
outliers(data, num_cols)

Больше выбросов содержат переменные decline_app_cnt, income и bki_request_cnt. 

In [ ]:
fig = plt.figure(figsize=[30, 8])
count = 1
for each in num_cols:
    for type_client in train.default.unique():
        ax = fig.add_subplot(1, 10, count)
        sns.boxplot(y=each,
                    data=train[(train.default == type_client)],
                    orient='v')
        ax.set_title(label=f'{each}, default={type_client}',
                     fontdict={'fontsize': 12})
        ax.set(ylabel='')
        plt.subplots_adjust(wspace=1)
        count += 1
plt.show()

Дефолтные клиенты по сравнение с недефолтными в среднем немного младше, имеют большее количество отмененных заявок, больше запросов в БКИ и более низкий доход и балл БКИ

In [ ]:
correlation = data[num_cols].corr()
fig, ax = plt.subplots(figsize=(10, 5))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap="BrBG",
            fmt='.2f', linewidths=.5, ax=ax, mask=matrix)

Между признаками нет высокой корреляции, поэтому (согласно корелляции Пирсона) они все могут быть включены в модель

### Корректировка непрерывных переменных

#### Логарифмирование числовых переменных

In [ ]:
for each in num_cols[:-1]:
    data[each] = np.log(data[each]+1)

#### Удаление шума

In [ ]:
del_outliers(data, num_cols)

In [ ]:
graph_out(data, num_cols)

### Формализация переменных типа object

In [ ]:
label_encoding(data)

## Работа с категориальными данными

In [ ]:
hist_type(data, bin_cols+cat_cols)

In [ ]:
correlation = data[num_cols + cat_cols].corr()
fig, ax = plt.subplots(figsize=(10, 5))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap="BrBG",
            fmt='.2f', linewidths=.5, ax=ax, mask=matrix)

Дублируются показатели home_address и work_address дублируются, sna и first_time. Есть небольшая зависмость между доходом и уровнем региона (возможно оценка рейтинга региона включает в себя средний доход жителей).

### Создание новых признаков и редактирование прежних

In [ ]:
data['app_date'] = data['app_date'].apply(
    lambda x: 0 if 0 <= x <= 4 else 1)  # 0 - будние дни, 1 - выходные дни

In [ ]:
data['education'] = data['education'].apply(lambda x: 2 if (x == 3) | (
    x == 6) | (x == 0) else x)  # 0, 2, 3, 6, группы объединить в группу 2

In [ ]:
data['education_lev'] = data['education'].copy()
data['region_rating_lev'] = data['region_rating'].copy()
data['home_address_lev'] = data['home_address'].copy()
data['sna_lev'] = data['sna'].copy()
data = pd.get_dummies(data, columns=['education_lev',
                                     'region_rating_lev',
                                     'home_address_lev', 'sna_lev'],
                      dummy_na=False, dtype='int64')

### Подготовка данных к моделированию

In [ ]:
# Разбиение данных не обучающую и тестовую выборку
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

In [ ]:
bin_cols.extend(['education_lev_1', 'education_lev_2',
                 'education_lev_4', 'education_lev_5',
                 'region_rating_lev_20', 'region_rating_lev_30',
                 'region_rating_lev_40', 'region_rating_lev_50',
                 'region_rating_lev_60', 'region_rating_lev_70',
                 'region_rating_lev_80', 'home_address_lev_1',
                 'home_address_lev_2','home_address_lev_3',
                 'sna_lev_1', 'sna_lev_2', 'sna_lev_3'])

In [ ]:
imp_num = pd.Series(f_classif(train_data[num_cols],
                              train_data['default'])[0],
                    index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Дисперсионный анализ (ANOVA) показывает, что наиболее значимым баллом является балл по БКИ, который, вероятно, былл расчитан как на основании возраста, дохода, количества отказов.
Поэтому возможно рассмотреть два варианта включения параметров: score_bki(1 вариант); decline_app_cnt, bki_request_cnt, income (2 вариант); age, decline_app_cnt, income, bki_request_cnt, score_bki (3 вариант). Первый и второй варианты моделирования не представлен в файле, так как целевые метрики ниже.  

In [ ]:
imp_cat = pd.Series(mutual_info_classif(train_data[bin_cols + cat_cols], train_data['default'],
                                        discrete_features=True), index=bin_cols + cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', figsize=(20, 10))

In [ ]:
correlation = data[bin_cols + cat_cols].corr()
fig, ax = plt.subplots(figsize=(30, 10))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap="BrBG",
            fmt='.2f', linewidths=.5, ax=ax, mask=matrix)

Показатели car и car_type дублируются. Согласно дисперсионному анализу (ANOVA) принято решение не включать в модель показатель car.
По данным корреляционного анализа показатели home_address и work_address дублируются. Согласно дисперсионному анализу (ANOVA) принято решение не включать в модель показатель work_address.
По данным корреляционного анализа показатели sna и first_time. Согласно дисперсионному анализу (ANOVA) принято решение не включать в модель показатель first_time.

Для модели выбраны переменные:

In [ ]:
# бинарные переменные
bin_cols = ['car_type', 'foreign_passport', 'education_lev_1',
            'education_lev_4', 'region_rating_lev_40',
            'region_rating_lev_80', 'home_address_lev_1',
            'home_address_lev_2','sna_lev_1']
# категориальные переменные
cat_cols = ['education', 'home_address', 'sna', 'region_rating']
# числовые переменные
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt', 'score_bki']

In [ ]:
y = train_data['default'].values
X = pd.concat([train_data[num_cols], train_data[cat_cols],
               train_data[bin_cols]], ignore_index=False, axis=1)

In [ ]:
train, validation = train_test_split(
    train_data, test_size=0.33, random_state=42)

In [ ]:
visualize_train_valid_counts(train_data, train, validation, 'default', 74800)

Выборка несбалансированная, следует сократить количество объектов превалирующего класса (default=0). опираться на метрику f1-score при анализе эффективности модели.

In [ ]:
# Стандартизация переменных
sscale = StandardScaler()
X_num_t = pd.DataFrame(sscale.fit_transform(train[num_cols].values))
X_num_v = pd.DataFrame(sscale.transform(validation[num_cols].values))
X_num_test = pd.DataFrame(sscale.transform(test_data[num_cols].values))


X_cat_t = train[cat_cols].values
X_cat_v = validation[cat_cols].values
X_cat_test = test_data[cat_cols].values

X_train = pd.DataFrame(np.hstack([X_num_t, train[bin_cols].values, X_cat_t]))
X_valid = pd.DataFrame(
    np.hstack([X_num_v, validation[bin_cols].values, X_cat_v]))
X_test = pd.DataFrame(
    np.hstack([X_num_test, test_data[bin_cols].values, X_cat_test]))

In [ ]:
y_train = train['default'].values
y_true = validation['default'].values

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_true.shape, X_test.shape

In [ ]:
scale = RobustScaler()
X_train = scale.fit_transform(X_train)
X_valid = scale.transform(X_valid)
X_test = scale.transform(X_test)

### Модель 1 (с параметрами по умолчанию)

In [ ]:
model_1 = LogisticRegression()
model_1.fit(X_train, y_train)
y_pred_1 = model_1.predict(X_valid)

In [ ]:
conf_mat = confusion_matrix(y_true, y_pred_1)
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true, y_pred_1)

In [ ]:
print(classification_report(y_true, y_pred_1))

In [ ]:
rocauc(y_true, y_pred_1)

#### Модель 2 (с подобранными наилучшими параметрами)

In [ ]:
y_pred_2 = best_pred(X_train, y_train, X_valid)[:, 1]

In [ ]:
conf_mat = confusion_matrix(y_true, np.round(y_pred_2))
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true, np.around(y_pred_2))

In [ ]:
print(classification_report(y_true, np.around(y_pred_2)))

In [ ]:
rocauc(y_true, y_pred_2)

### Модель 3 (подбор параметров через optuna в sklearn)

In [ ]:
'''

Параметры логистической модели устанавливаются вручную из предыдущей модели.
Это связано с тем, что model.set_params, где параметр - результат реализации другой функции,
занимает много времени.

'''


def objective(trial):

    param_C = trial.suggest_float('C', 0.0, 1.0)
    
    model = LogisticRegression(C=param_C, class_weight='balanced',
                               dual=False, fit_intercept=True,
                               intercept_scaling=1, l1_ratio=None,
                               max_iter=1000, multi_class='ovr',
                               n_jobs=None, penalty='l2',
                               random_state=None, solver='sag',
                               tol=0.001, verbose=0, warm_start=False)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    f1 = f1_score(y_true, y_pred)

    return f1

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
best_с = study.best_params

In [ ]:
model_3 = LogisticRegression(**best_с, class_weight='balanced',
                             dual=False, fit_intercept=True,
                             intercept_scaling=1, l1_ratio=None,
                             max_iter=1000, multi_class='ovr',
                             n_jobs=None, penalty='l2',
                             random_state=None, solver='sag',
                             tol=0.001, verbose=0, warm_start=False)
model_3.fit(X_train, y_train)
y_pred_3 = model_3.predict(X_valid)

In [ ]:
conf_mat = confusion_matrix(y_true, np.round(y_pred_3))
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true, y_pred_3)

In [ ]:
rocauc(y_true, y_pred_3)

## Undersampling

In [ ]:
X_us = train_data.loc[:, train_data.columns != 'default']
y_us = pd.DataFrame({'default': train_data['default']})

In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy='majority')
X_rus, y_rus = rus.fit_resample(X_us, y_us)

In [ ]:
X_rus.shape, y_rus.shape

In [ ]:
data_rus = pd.concat([X_rus, y_rus], ignore_index=False, axis=1)

In [ ]:
data_rus.info()

In [ ]:
train_rus, validation_rus = train_test_split(
    data_rus, test_size=0.33, random_state=42)

In [ ]:
visualize_train_valid_counts(
    data_rus, train_rus, validation_rus, 'default', 10000)

Разбиение сбалансировано

In [ ]:
# Стандартизация переменных
sscale = StandardScaler()

X_num_t_rus = pd.DataFrame(
    sscale.fit_transform(train_rus[num_cols].values))
X_num_v_rus = pd.DataFrame(
    sscale.transform(validation_rus[num_cols].values))

X_cat_t_rus = train_rus[cat_cols].values
X_cat_v_rus = validation_rus[cat_cols].values

X_train_rus = pd.DataFrame(
    np.hstack([X_num_t_rus, train_rus[bin_cols].values, X_cat_t_rus]))
X_valid_rus = pd.DataFrame(
    np.hstack([X_num_v_rus, validation_rus[bin_cols].values, X_cat_v_rus]))

In [ ]:
y_train_rus = train_rus['default'].values
y_true_rus = validation_rus['default'].values

In [ ]:
X_train_rus.shape, X_valid_rus.shape, y_train_rus.shape, y_true_rus.shape

In [ ]:
scale = RobustScaler()
X_train_rus = scale.fit_transform(X_train_rus)
X_valid_rus = scale.transform(X_valid_rus)

### Модель 4. (с параметрами по умолчанию)

In [ ]:
model_4 = LogisticRegression()
model_4.fit(X_train_rus, y_train_rus)
y_pred_4 = model_4.predict(X_valid_rus)

In [ ]:
conf_mat = confusion_matrix(y_true_rus, y_pred_4)
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true_rus, y_pred_4)

In [ ]:
print(classification_report(y_true_rus, y_pred_4))

In [ ]:
rocauc(y_true_rus, y_pred_4)

#### Модель 5 (с подобранными наилучшими параметрами)

In [ ]:
y_pred_5 = best_pred(X_train_rus, y_train_rus, X_valid_rus)[:, 1]

In [ ]:
conf_mat = confusion_matrix(y_true_rus, np.around(y_pred_5))
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true_rus, np.around(y_pred_5))

In [ ]:
print(classification_report(y_true_rus, np.around(y_pred_5)))

In [ ]:
rocauc(y_true_rus, y_pred_5)

### Модель 6 (подбор параметров через optuna в sklearn)

In [ ]:
'''

Параметры логистической модели устанавливаются вручную из предыдущей модели.
Это связано с тем, что model.set_params, где параметр - результат реализации другой функции,
занимает много времени.

'''


def objective(trial):

    param_C = trial.suggest_float('C', 0.0, 1.0)
    model = LogisticRegression(C=param_C, class_weight='balanced',
                               dual=False, fit_intercept=True,
                               intercept_scaling=1, l1_ratio=None,
                               max_iter=1000, multi_class='ovr',
                               n_jobs=None, penalty='l1',
                               random_state=None, solver='liblinear',
                               tol=0.001, verbose=0, warm_start=False)
    model.fit(X_train_rus, y_train_rus)
    y_pred = model.predict(X_valid_rus)
    f1 = f1_score(y_true_rus, y_pred)

    return f1

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
best_с = study.best_params

In [ ]:
best_с

In [ ]:
model_6 = LogisticRegression(**best_с, class_weight='balanced',
                             dual=False, fit_intercept=True,
                             intercept_scaling=1, l1_ratio=None,
                             max_iter=1000, multi_class='ovr',
                             n_jobs=None, penalty='l1',
                             random_state=None, solver='liblinear',
                             tol=0.001, verbose=0, warm_start=False)
model_6.fit(X_train_rus, y_train_rus)
y_pred_6 = model_6.predict(X_valid_rus)

In [ ]:
conf_mat = confusion_matrix(y_true_rus, y_pred_6)
print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
print_logisitc_metrics(y_true_rus, y_pred_6)

In [ ]:
print(classification_report(y_true_rus, np.around(y_pred_6)))

In [ ]:
rocauc(y_true_rus, y_pred_6)

Первые три модели построены на несбалансированных данных, что и показывает матрица ошибок, в которой значение True Negative значительно выше остальных. Это объясняется тем, что в несбалансированных данных больше информации о недефолтных клиентах. Это обусловило то, что модель научилась в большинстве случаев прогнозировать "недефолт". Последние три модели было построены после сокращения количества объектов превалирующих классов (undersampling).  Для выбора модели единственным параметром выбран **f-score**, как сочетание между  precision и recall. f-score показывает, что качество **6 модель** наиболее адекватна.

### Submission

In [ ]:
model_final = LogisticRegression(**best_с, class_weight='balanced',
                                 dual=False, fit_intercept=True,
                                 intercept_scaling=1, l1_ratio=None,
                                 max_iter=1000, multi_class='ovr',
                                 n_jobs=None, penalty='l1',
                                 random_state=42, solver='liblinear',
                                 tol=0.001, verbose=0, warm_start=False)
model_final.fit(X_train, y_train)

In [ ]:
predict_submission = model_final.predict(X_test)

In [ ]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission.head(10)